# 数据结构之排序
## 第五章 外部排序

　　外部排序算法，即要借助外部存储器对数据进行排序的算法，包括置换平衡归并排序算法、置换选择排序算法等。

　　外部排序算法的实现，其实就是将体积大的数据分割为内存容得下的多份数据，然后分别使用内部排序算法进行排序，最后进行整合。

　　和内部排序算法不同，外部排序算法的主要影响因素在于读写内存的次数。

## 1. 什么是外部排序算法

　　上一章介绍了很多排序算法，插入排序、选择排序、归并排序等等，这些算法都属于内部排序算法，即排序的整个过程只是在内存中完成。而当待排序的文件比内存的可使用容量还大时，文件无法一次性放到内存中进行排序，需要借助于外部存储器（例如硬盘、U盘、光盘），这时就需要用到本章介绍的外部排序算法来解决。
　　
　　外部排序算法由两个阶段构成：
　（1）按照内存大小，将大文件分成若干长度为 l 的子文件（l 应小于内存的可使用容量），然后将各个子文件依次读入内存，使用适当的内部排序算法对其进行排序（排好序的子文件统称为“归并段”或者“顺段”），将排好序的归并段重新写入外存，为下一个子文件排序腾出内存空间；
　（2）对得到的顺段进行合并，直至得到整个有序的文件为止。
 　　
　　例如，有一个含有 10000 个记录的文件，但是内存的可使用容量仅为 1000 个记录，毫无疑问需要使用外部排序算法，具体分为两步：
　（1）将整个文件其等分为 10 个临时文件（每个文件中含有 1000 个记录），然后将这 10 个文件依次进入内存，采取适当的内存排序算法对其中的记录进行排序，将得到的有序文件（初始归并段）移至外存。
　（2）对得到的 10 个初始归并段进行如图 1 的两两归并，直至得到一个完整的有序文件。
 
　【注意：此例中采用了将文件进行等分的操作，还有不等分的算法，后面章节会介绍】

![Image Name](https://cdn.kesci.com/upload/image/qxao2n954y.png?imageView2/0/w/640/h/640)

　　如图 1 所示有 10 个初始归并段到一个有序文件，共进行了 4 次归并，每次都由 m 个归并段得到 ⌈m/2⌉ 个归并段，这种归并方式被称为 **2-路平衡归并**。

　【注意：在实际归并的过程中，由于内存容量的限制不能满足同时将 2 个归并段全部完整的读入内存进行归并，只能不断地取 2 个归并段中的每一小部分进行归并，通过不断地读数据和向外存写数据，直至 2 个归并段完成归并变为 1 个大的有序文件。】
 
 　　**对于外部排序算法来说，影响整体排序效率的因素主要取决于读写外存的次数，即访问外存的次数越多，算法花费的时间就越多，效率就越低。**
 
 　【计算机中处理数据的为中央处理器（CPU），如若需要访问外存中的数据，只能通过将数据从外存导入内存，然后从内存中获取。同时由于内存读写速度快，外存读写速度慢的差异，更加影响了外部排序的效率。】

 　　**对于同一个文件来说，对其进行外部排序时访问外存的次数同归并的次数成正比，即归并操作的次数越多，访问外存的次数就越多。**图 1 中使用的是 2-路平衡归并的方式，举一反三，还可以使用 3-路归并、4-路归并甚至是 10-路归并的方式，图 2 为 5-路归并的方式：

![Image Name](https://cdn.kesci.com/upload/image/qxaoo9gusx.png?imageView2/0/w/640/h/640)
	 
 　　对比图1和图2可以看出，对于 k-路平衡归并中 k 值得选择，增加 k 可以减少归并的次数，从而减少外存读写的次数，最终达到提高算法效率的目的。**除此之外，一般情况下对于具有 m 个初始归并段进行 k-路平衡归并时，归并的次数为：s=⌊logk⁡m ⌋（其中 s 表示归并次数）。**
	 
　　从公式上可以判断出，想要达到减少归并次数从而提高算法效率的目的，可以从两个角度实现：
　（1）增加 k-路平衡归并中的 k 值；
　（2）尽量减少初始归并段的数量 m，即增加每个归并段的容量；
 
 　【其增加 k 值的想法引申出了一种外部排序算法：多路平衡归并算法；增加数量 m 的想法引申出了另一种外部排序算法：置换-选择排序算法。两种外部排序算法会在后序章节中详细介绍。】

## 2. 多路平衡归并排序（胜者树、败者树）

　　通过上一节对于外部排序的介绍得知：对于外部排序算法来说，其直接影响算法效率的因素为读写外存的次数，即次数越多，算法效率越低。若想提高算法的效率，即减少算法运行过程中读写外存的次数，可以增加 k –路平衡归并中的 k 值。
	
　　**但是经过计算得知，如果毫无限度地增加 k 值，虽然会减少读写外存数据的次数，但会增加内部归并的时间，得不偿失。**
	
　　例如在上节中，对于 10 个临时文件，当采用 2-路平衡归并时，若每次从 2 个文件中想得到一个最小值时只需比较 1 次；而采用 5-路平衡归并时，若每次从 5 个文件中想得到一个最小值就需要比较 4 次。以上仅仅是得到一个最小值记录，如要得到整个临时文件，其耗费的时间就会相差很大。
	
　　**为了避免在增加 k 值的过程中影响内部归并的效率，在进行 k-路归并时可以使用“败者树”来实现，该方法在增加 k 值时不会影响其内部归并的效率。**


- 败者树实现内部归并

　　败者树是树形选择排序的一种变形，本身是一棵完全二叉树。
	
　　在树形选择排序一节中，对于无序表{49，38，65，97，76，13，27，49}创建的完全二叉树如图 1 所示，构建此树的目的是选出无序表中的最小值。

　　这棵树与败者树正好相反，是一棵“**胜者树**”。因为树中每个非终端结点（除叶子结点之外的其它结点）中的值都表示的是左右孩子相比较后的较小值（谁最小即为胜者）。例如叶子结点 49 和 38 相对比，由于 38 更小，所以其双亲结点中的值保留的是胜者 38。然后用 38 去继续同上层去比较，一直比较到树的根结点。

![Image Name](https://cdn.kesci.com/upload/image/qxapzjz3wa.png?imageView2/0/w/640/h/640)

　　**而败者树恰好相反，其双亲结点存储的是左右孩子比较之后的失败者，而胜利者则继续同其它的胜者去比较。**

　　例如还是图 1 中，叶子结点 49 和 38 比较，38 更小，所以 38 是胜利者，49 为失败者，但由于是败者树，所以其双亲结点存储的应该是 49；同样，叶子结点 65 和 97 比较，其双亲结点中存储的是 97 ，而 65 则用来同 38 进行比较，65 会存储到 97 和 49 的双亲结点的位置，38 继续做后续的胜者比较，依次类推。
胜者树和败者树的区别就是：胜者树中的非

　【胜者树和败者树的区别就是：胜者树中的非终端结点中存储的是胜利的一方；而败者树中的非终端结点存储的是失败的一方。而在比较过程中，都是拿胜者去比较。】

![Image Name](https://cdn.kesci.com/upload/image/qxaq2a49gh.png?imageView2/0/w/640/h/640)

　　如图 2 所示为一棵 5-路归并的败者树，其中 b0—b4 为树的叶子结点，分别为 5 个归并段中存储的记录的关键字。 ls 为一维数组，表示的是非终端结点，其中存储的数值表示第几归并段（例如 b0 为第 0 个归并段）。ls[0] 中存储的为最终的胜者，表示当前第 3 归并段中的关键字最小。
	
　　当最终胜者判断完成后，只需要更新叶子结点 b3 的值，即导入关键字 15，然后让该结点不断同其双亲结点所表示的关键字进行比较，败者留在双亲结点中，胜者继续向上比较。
	
　　例如，叶子结点 15 先同其双亲结点 ls[4] 中表示的 b4 中的 12 进行比较，12 为胜利者，则 ls[4] 改为 15，然后 12 继续同 ls[2] 中表示的 10 做比较，10 为胜者，然后 10 继续同其双亲结点 ls[1] 表示的 b1（关键字 9）作比较，最终 9 为胜者。整个过程如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxaq44n9zf.png?imageView2/0/w/640/h/640)

　【注意：为了防止在归并过程中某个归并段变为空，处理的办法为：可以在每个归并段最后附加一个关键字为最大值的记录。这样当某一时刻选出的冠军为最大值时，表明 5 个归并段已全部归并完成。（因为只要还有记录，最终的胜者就不可能是附加的最大值）】

## 3. 置换选择排序

　　上一节介绍了增加 k-路归并排序中的 k 值来提高外部排序效率的方法，而除此之外，还有另外一条路可走，即减少初始归并段的个数，也就是本章第一节中提到的减小 m 的值。
	
　【m 的求值方法为：m=⌈n/l⌉（n 表示为外部文件中的记录数，l 表示初始归并段中包含的记录数）】
 
　　如果要想减小 m 的值，在外部文件总的记录数 n 值一定的情况下，只能增加每个归并段中所包含的记录数 l。而对于初始归并段的形成，就不能再采用上一章所介绍的内部排序的算法，因为所有的内部排序算法正常运行的前提是所有的记录都存在于内存中，而内存的可使用空间是一定的，如果增加 l 的值，内存是盛不下的。
	
　　所以要另想它法，探索一种新的排序方法：置换—选择排序算法。
	
　　例如已知初始文件中总共有 24 个记录，假设内存工作区最多可容纳 6 个记录，按照之前的选择排序算法最少也只能分为 4 个初始归并段。而如果使用置换—选择排序，可以实现将 24 个记录分为 3 个初始归并段，如图 1 所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxaq9ci5o0.png?imageView2/0/w/640/h/640)

　　置换—选择排序算法的具体操作过程为：

　　1. 首先从初始文件中输入 6 个记录到内存工作区中；
	
　　2. 从内存工作区中选出关键字最小的记录，将其记为 MINIMAX 记录；
	
　　3. 然后将 MINIMAX 记录输出到归并段文件中；
	
　　4. 此时内存工作区中还剩余 5 个记录，若初始文件不为空，则从初始文件中输入下一个记录到内存工作区中；
	
　　5. 从内存工作区中的所有比 MINIMAX 值大的记录中选出值最小的关键字的记录，作为新的 MINIMAX 记录；
	
　　6. 重复过程 3—5，直至在内存工作区中选不出新的 MINIMAX 记录为止，由此就得到了一个初始归并段；
	
　　7. 重复 2—6，直至内存工作为空，由此就可以得到全部的初始归并段。

　　拿图 1 中的初始文件为例，首先输入前 6 个记录到内存工作区，其中关键字最小的为 29，所以选其为 MINIMAX 记录，同时将其输出到归并段文件中，如下图所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxaqvj717q.png?imageView2/0/w/640/h/320)

　　此时初始文件不为空，所以从中输入下一个记录 14 到内存工作区中，然后从内存工作区中的比 29 大的记录中，选择一个最小值作为新的 MINIMAX 值输出到 归并段文件中，如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxaqxduvpy.png?imageView2/0/w/640/h/320)

　　初始文件还不为空，所以继续输入 61 到内存工作区中，从内存工作区中的所有关键字比 38 大的记录中，选择一个最小值作为新的 MINIMAX 值输出到归并段文件中，如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxaqyl2ss0.png?imageView2/0/w/640/h/320)

　　如此重复性进行，直至选不出 MINIMAX 值为止，如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxar09fdol.png?imageView2/0/w/640/h/640)

　　当选不出 MINIMAX 值时，表示一个归并段已经生成，则开始下一个归并段的创建，创建过程同第一个归并段一样，这里不再赘述。

　　在上述创建初始段文件的过程中，需要不断地在内存工作区中选择新的 MINIMAX 记录，即选择不小于旧的 MINIMAX 记录的最小值，此过程需要利用“败者树”来实现。
	
　　同上一节所用到的败者树不同的是，在不断选择新的 MINIMAX 记录时，为了防止新加入的关键字值小的的影响，每个叶子结点附加一个序号位，当进行关键字的比较时，先比较序号，序号小的为胜者；序号相同的关键字值小的为胜者。

　　在初期创建败者树时也可以通过不断调整败者树的方式，其中所有记录的序号均设为 0 ，然后从初始文件中逐个输入记录到内存工作区中，自下而上调整败者树。过程如下：

　　1. 首先创建一个空的败者树，如下图所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxar3vj8g1.png?imageView2/0/w/640/h/640)

　【提示：败者树根结点上方的方框内表示的为最终的胜者所处的位置。】
	
　　2. 从初始文件中读入关键字为 51 的记录，自下往上调整败者树，如下图所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxar5iyoys.png?imageView2/0/w/640/h/640)

　【提示：序号 1 默认为比 0 小，为败者。】
 
　　3. 从初始文件中读入关键字为 49 的记录，调整败者树如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxar79hvd3.png?imageView2/0/w/640/h/640)

　　4. 从初始文件依次读入关键字为 39、46、38、29 的记录，调整败者树如下图所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxar8kbfwz.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qxar9on1gf.png?imageView2/0/w/640/h/640)

　　由败者树得知，其最终胜者为 29，设为 MINIMAX 值，将其输出到初始归并文件中，同时再读入下一个记录 14，调整败者树，如下图所示：

![Image Name](https://cdn.kesci.com/upload/image/qxarjhzooq.png?imageView2/0/w/640/h/640)

　【**注意：**当读入新的记录时，如果其值比 MINIMAX 大，其序号则仍为 1；反之则为 2 ，比较时序号 1 比序号 2的记录大。】

　　通过不断地向败者树中读入记录，会产生多个 MINIMAX，直到最终所有叶子结点中的序号都为 2，此时产生的新的 MINIMAX 值的序号 2，表明此归并段生成完成，而此新的 MINIMAX 值就是下一个归并段中的第一个记录。

## 4. 最佳归并树

　　通过上一节对置换-选择排序算法的学习了解到，通过对初始文件进行置换选择排序能够获得多个长度不等的初始归并段，相比于按照内存容量大小对初始文件进行等分，大大减少了初始归并段的数量，从而提高了外部排序的整体效率。

　　**本节带领大家思考一个问题：无论是通过等分还是置换-选择排序得到的归并段，如何设置它们的归并顺序，可以使得对外存的访问次数降到最低？**

　　例如，现有通过置换选择排序算法所得到的 9 个初始归并段，其长度分别为：9，30，12，18，3，17，2，6，24。在对其采用 3-路平衡归并的方式时可能出现如图 1 所示的情况：

![Image Name](https://cdn.kesci.com/upload/image/qxarnzi5dh.png?imageView2/0/w/640/h/640)

　【提示：图 1 中的叶子结点表示初始归并段，各自包含记录的长度用结点的权重来表示；非终端结点表示归并后的临时文件。】
 
　　假设在进行平衡归并时，操作每个记录都需要单独进行一次对外存的读写，那么图 1 中的归并过程需要对外存进行读或者写的次数为：
	
　【（9+30+12+18+3+17+2+6+24）*2*2=484（图 1 中涉及到了两次归并，对外存的读和写各进行 2 次）】
	
　　从计算结果上看，对于图 1 中的 3 叉树来讲，其操作外存的次数恰好是树的带权路径长度的 2 倍。所以，对于如何减少访问外存的次数的问题，就等同于考虑如何使 k-路归并所构成的 k 叉树的带权路径长度最短。
	
　　若想使树的带权路径长度最短，就是**构造赫夫曼树**。
	
　【在学习赫夫曼树时，只是涉及到了带权路径长度最短的二叉树为赫夫曼树，其实扩展到一般情况，对于 k 叉树，只要其带权路径长度最短，亦可以称为赫夫曼树。】

　　若对上述 9 个初始归并段构造一棵赫夫曼树作为归并树，如图 2 所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qxb014sr7c.png?imageView2/0/w/640/h/640)

　　依照图 2 所示，其对外存的读写次数为：	(2*3+3*3+6*3+9*2+12*2+17*2+18*2+24*2+30)*2=446
	
　　通过以构建赫夫曼树的方式构建归并树，使其对读写外存的次数降至最低（k-路平衡归并，需要选取合适的 k 值，构建赫夫曼树作为归并树）。所以称此归并树为最佳归并树。
	

- 附加“虚段”的归并树

　　上述图 2 中所构建的为一颗真正的 3叉树（树中各结点的度不是 3 就是 0），而若 9 个初始归并段改为 8 个，在做 3-路平衡归并的时候就需要有一个结点的度为 2。
	
　　对于具体设置哪个结点的度为 2，为了使总的带权路径长度最短，正确的选择方法是：**附加一个权值为 0 的结点（称为“虚段”），然后再构建赫夫曼树**。例如图 2 中若去掉权值为 30 的结点，其附加虚段的最佳归并树如图 3 所示：

![Image Name](https://cdn.kesci.com/upload/image/qxb05437gx.png?imageView2/0/w/640/h/640)

　【注意：虚段的设置只是为了方便构建赫夫曼树，在构建完成后虚段自动去掉即可。】
	
　　对于如何判断是否需要增加虚段，以及增加多少虚段的问题，有以下结论直接套用即可：

　　在一般情况下，对于 k–路平衡归并来说，若 (m-1)MOD(k-1)=0，则不需要增加虚段；否则需附加 k-(m-1)MOD(k-1)-1 个虚段。
	
　　本章用了 4 节的内容介绍了实现外部排序的两个过程：
　（1）将整个初始文件分为多个初始归并段;
　（2）将初始归并段进行归并，直至得到一个有序的完整文件；
 
 
 　　为了提高整个外部排序的效率，本章分别从以上两个方面对外部排序进行了优化：
　（1）在实现将初始文件分为 m 个初始归并段时，为了尽量减小 m 的值，采用置换-选择排序算法，可实现将整个初始文件分为数量较少的长度不等的初始归并段。
　（2）同时在将初始归并段归并为有序完整文件的过程中，为了尽量减少读写外存的次数，采用构建最佳归并树的方式，对初始归并段进行归并，而归并的具体实现方法是采用败者树的方式。